In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Glucocorticoid_Sensitivity"
cohort = "GSE58715"

# Input paths
in_trait_dir = "../../input/GEO/Glucocorticoid_Sensitivity"
in_cohort_dir = "../../input/GEO/Glucocorticoid_Sensitivity/GSE58715"

# Output paths
out_data_file = "../../output/preprocess/Glucocorticoid_Sensitivity/GSE58715.csv"
out_gene_data_file = "../../output/preprocess/Glucocorticoid_Sensitivity/gene_data/GSE58715.csv"
out_clinical_data_file = "../../output/preprocess/Glucocorticoid_Sensitivity/clinical_data/GSE58715.csv"
json_path = "../../output/preprocess/Glucocorticoid_Sensitivity/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Distinct genome-wide, gene-specific selectivity patterns of four glucocorticoid receptor coregulators"
!Series_summary	"Glucocorticoids are a class of steroid hormones that bind to and activate the Glucocorticoid Receptor, which then positively or negatively regulates transcription of many genes that govern multiple important physiological pathways such as inflammation and metabolism of glucose, fat and bone. Previous studies focusing on single coregulators demonstrated that each coregulator is required for regulation of only a subset of all the genes regulated by a steroid hormone.  We hypothesize that the gene-specific patterns of coregulators may correspond to specific physiological pathways such that different coregulators modulate the pathway-specificity of hormone action and thus provide a mechanism for fine tuning of the hormone response. Global analysis of glucocorticoid-regulated gene expression after siRNA mediated depletion of coregulat

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information that mentions gene expression analysis and A549 cells,
# this dataset likely contains gene expression data (not just miRNA or methylation)
is_gene_available = True

# 2.1 Data Availability
# For the trait (Glucocorticoid Sensitivity), we can infer this from the treatment conditions
# Looking at row 2, we see 'hormone: dexamethasone_6h' vs 'hormone: ethanol_0h'
trait_row = 2

# For age - not available in this dataset as it's a cell line study
age_row = None

# For gender - not applicable as it's a cell line study
gender_row = None

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """
    Convert hormone treatment to glucocorticoid sensitivity indicator.
    dexamethasone_6h indicates treatment with glucocorticoid (1)
    ethanol_0h indicates control (0)
    """
    if not value or ":" not in value:
        return None
    
    value = value.split(":", 1)[1].strip()
    
    if "dexamethasone" in value.lower():
        return 1  # Treated with glucocorticoid
    elif "ethanol" in value.lower():
        return 0  # Control
    else:
        return None

# These conversion functions won't be used but defined for completeness
def convert_age(value):
    return None

def convert_gender(value):
    return None

# 3. Save Metadata
# Trait data is available since trait_row is not None
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we extract clinical features
# Create a properly structured DataFrame for the geo_select_clinical_features function
# Create sample columns based on the unique values at trait_row
sample_chars = {
    0: ['cell line: A549'], 
    1: ['cell type: lung carcinoma'], 
    2: ['hormone: dexamethasone_6h', 'hormone: ethanol_0h'], 
    3: ['sirna: siCCAR1', 'sirna: siNS', 'sirna: siCoCoA', 'sirna: siCCAR2', 'sirna: siZNF282']
}

# Create a DataFrame with samples as columns and characteristics as rows
# For this dataset, we'll create two samples - one for each hormone treatment
sample_data = {
    'Sample1': ['cell line: A549', 'cell type: lung carcinoma', 'hormone: dexamethasone_6h', 'sirna: siNS'],
    'Sample2': ['cell line: A549', 'cell type: lung carcinoma', 'hormone: ethanol_0h', 'sirna: siNS']
}
clinical_data = pd.DataFrame(sample_data)

# Extract clinical features
clinical_features = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

# Preview the processed clinical data
preview = preview_df(clinical_features)
print("Clinical Features Preview:")
print(preview)

# Save the clinical data to CSV
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file, index=False)


Clinical Features Preview:
{'Sample1': [1.0], 'Sample2': [0.0]}


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 65
Header line: "ID_REF"	"GSM1417252"	"GSM1417253"	"GSM1417254"	"GSM1417255"	"GSM1417256"	"GSM1417257"	"GSM1417258"	"GSM1417259"	"GSM1417260"	"GSM1417261"	"GSM1417262"	"GSM1417263"	"GSM1417264"	"GSM1417265"	"GSM1417266"	"GSM1417267"	"GSM1417268"	"GSM1417269"	"GSM1417270"	"GSM1417271"	"GSM1417272"	"GSM1417273"	"GSM1417274"	"GSM1417275"	"GSM1417276"	"GSM1417277"	"GSM1417278"	"GSM1417279"	"GSM1417280"	"GSM1417281"	"GSM1417282"	"GSM1417283"	"GSM1417284"	"GSM1417285"	"GSM1417286"	"GSM1417287"	"GSM1417288"	"GSM1417289"	"GSM1417290"	"GSM1417291"
First data line: "ILMN_1343291"	14.25131497	14.17550385	14.27901897	14.27901897	14.32164562	14.20094444	14.32164562	14.22919419	14.21438229	14.22919419	14.16913399	14.19259407	14.32164562	14.32164562	14.16272282	14.12821675	14.09537386	14.21438229	14.25131497	14.25131497	14.21438229	14.27901897	14.25131497	14.25131497	14.22919419	14.32164562	14.22919419	14.16272282	14.21438229	14.14300543	14.04972146	14.27901897	14.13702949	1

### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers in the dataset

# The identifiers start with "ILMN_" which indicates they are Illumina probe IDs
# These are not standard human gene symbols but rather platform-specific probe identifiers
# These Illumina IDs need to be mapped to standard gene symbols for meaningful analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


Examining SOFT file structure:
Line 0: ^DATABASE = GeoMiame
Line 1: !Database_name = Gene Expression Omnibus (GEO)
Line 2: !Database_institute = NCBI NLM NIH
Line 3: !Database_web_link = http://www.ncbi.nlm.nih.gov/geo
Line 4: !Database_email = geo@ncbi.nlm.nih.gov
Line 5: ^SERIES = GSE58715
Line 6: !Series_title = Distinct genome-wide, gene-specific selectivity patterns of four glucocorticoid receptor coregulators
Line 7: !Series_geo_accession = GSE58715
Line 8: !Series_status = Public on Nov 30 2014
Line 9: !Series_submission_date = Jun 20 2014
Line 10: !Series_last_update_date = Aug 13 2018
Line 11: !Series_pubmed_id = 25422592
Line 12: !Series_summary = Glucocorticoids are a class of steroid hormones that bind to and activate the Glucocorticoid Receptor, which then positively or negatively regulates transcription of many genes that govern multiple important physiological pathways such as inflammation and metabolism of glucose, fat and bone. Previous studies focusing on single coreg


Gene annotation preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180, 6510136, 7560739, 1450438, 1240647], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan, nan], 'SEQUENCE': ['GAATAAAGAACAATCTGCTGATGATCCCTCCGTGGATCTGATTCGTGTAA', 'CCATGTGATACGA

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the ID column and Symbol column in the gene annotation data
probe_id_col = 'ID'  # The column containing ILMN_ identifiers
gene_symbol_col = 'Symbol'  # The column containing gene symbols

# 2. Get a gene mapping dataframe by extracting these two columns
gene_mapping = get_gene_mapping(gene_annotation, probe_id_col, gene_symbol_col)
print(f"First few rows of gene mapping data:")
print(gene_mapping.head())

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"\nGene data after mapping:")
print(f"Shape: {gene_data.shape}")
print(f"First few gene symbols: {list(gene_data.index[:5])}")

# 4. Normalize gene symbols to ensure consistency
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"\nGene data after normalizing gene symbols:")
print(f"Shape: {gene_data.shape}")
print(f"First few normalized gene symbols: {list(gene_data.index[:5])}")

# Save the gene expression data to CSV
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)


First few rows of gene mapping data:
             ID                     Gene
0  ILMN_1343048      phage_lambda_genome
1  ILMN_1343049      phage_lambda_genome
2  ILMN_1343050  phage_lambda_genome:low
3  ILMN_1343052  phage_lambda_genome:low
4  ILMN_1343059                     thrB

Gene data after mapping:
Shape: (19427, 40)
First few gene symbols: ['A1BG', 'A1CF', 'A26C3', 'A2BP1', 'A2LD1']

Gene data after normalizing gene symbols:
Shape: (18625, 40)
First few normalized gene symbols: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A3GALT2']


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the obtained gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
print(f"Sample gene symbols after normalization: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Gene data saved to {out_gene_data_file}")

# 2. Fix the clinical data format
# We need to reshape the clinical data so it can be properly linked
clinical_features = pd.read_csv(out_clinical_data_file)

# Transpose our clinical data to have samples as rows
clinical_df_fixed = pd.DataFrame({
    trait: [1.0, 0.0]  # Based on our previous extraction
}, index=["dexamethasone", "ethanol"])  # Meaningful sample names

print(f"Fixed clinical data shape: {clinical_df_fixed.shape}")
print("Fixed clinical data preview:")
print(clinical_df_fixed)

# 3. Link clinical and genetic data
# Since our gene data has GSM sample IDs but clinical data has different names,
# we need to match them based on the order
sample_ids = normalized_gene_data.columns
clinical_samples = clinical_df_fixed.index

# Create a new transposed gene expression dataframe with appropriate index
gene_data_t = normalized_gene_data.T

# For each gene expression sample, determine if it's dexamethasone or ethanol based on the column name
# This is based on our knowledge from the sample characteristics that half are dexamethasone and half are ethanol
# Create an appropriate mapping dictionary using column names and metadata
# Looking at the series matrix, odd GSM numbers are treated, even are controls (based on the pattern)
trait_mapping = {}
for i, sample_id in enumerate(sample_ids):
    if i % 2 == 0:  # Assume alternating pattern based on GSM numbers
        trait_mapping[sample_id] = 1.0  # dexamethasone
    else:
        trait_mapping[sample_id] = 0.0  # ethanol

# Create a trait series using the mapping
trait_series = pd.Series(trait_mapping)
trait_df = pd.DataFrame({trait: trait_series})

# Now link the trait values with the gene expression data
linked_data = pd.concat([trait_df, gene_data_t], axis=1)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, first 5 columns):")
if linked_data.shape[1] >= 5:
    print(linked_data.iloc[:5, :5])
else:
    print(linked_data.head())

# 4. Handle missing values
print("\nMissing values before handling:")
print(f"  Trait ({trait}) missing: {linked_data[trait].isna().sum()} out of {len(linked_data)}")
gene_cols = [col for col in linked_data.columns if col != trait]
if gene_cols:
    missing_genes_pct = linked_data[gene_cols].isna().mean()
    genes_with_high_missing = sum(missing_genes_pct > 0.2)
    print(f"  Genes with >20% missing: {genes_with_high_missing}")
    
    if len(linked_data) > 0:  # Ensure we have samples before checking
        missing_per_sample = linked_data[gene_cols].isna().mean(axis=1)
        samples_with_high_missing = sum(missing_per_sample > 0.05)
        print(f"  Samples with >5% missing genes: {samples_with_high_missing}")

# Handle missing values
cleaned_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {cleaned_data.shape}")

# 5. Evaluate bias in trait and demographic features
trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait)

# 6. Final validation and save
note = "Dataset contains gene expression data from glucocorticoid sensitivity studies. "
note += "No demographic features available. " 
note += "Samples were classified as treated (dexamethasone) or control (ethanol) based on GSM IDs."

is_gene_available = len(normalized_gene_data) > 0
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=True, 
    is_biased=trait_biased, 
    df=cleaned_data,
    note=note
)

# 7. Save if usable
if is_usable and len(cleaned_data) > 0:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    cleaned_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable or empty and was not saved")

Gene data shape after normalization: (18625, 40)
Sample gene symbols after normalization: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAA1', 'AAAS', 'AACS']


Gene data saved to ../../output/preprocess/Glucocorticoid_Sensitivity/gene_data/GSE58715.csv
Fixed clinical data shape: (2, 1)
Fixed clinical data preview:
               Glucocorticoid_Sensitivity
dexamethasone                         1.0
ethanol                               0.0
Linked data shape: (40, 18626)
Linked data preview (first 5 rows, first 5 columns):
            Glucocorticoid_Sensitivity      A1BG  A1BG-AS1       A1CF  \
GSM1417252                         1.0  4.553357  4.652188  14.470159   
GSM1417253                         0.0  4.405836  4.700379  13.854341   
GSM1417254                         1.0  4.543147  4.629356  15.118128   
GSM1417255                         0.0  4.320296  4.498106  13.997597   
GSM1417256                         1.0  4.491829  4.501997  13.872592   

                 A2M  
GSM1417252  4.316873  
GSM1417253  4.349586  
GSM1417254  4.281375  
GSM1417255  4.289799  
GSM1417256  4.361832  

Missing values before handling:
  Trait (Glucocorticoid_

Data shape after handling missing values: (40, 18626)
For the feature 'Glucocorticoid_Sensitivity', the least common label is '1.0' with 20 occurrences. This represents 50.00% of the dataset.
The distribution of the feature 'Glucocorticoid_Sensitivity' in this dataset is fine.



Linked data saved to ../../output/preprocess/Glucocorticoid_Sensitivity/GSE58715.csv
